<a href="https://colab.research.google.com/github/hululuzhu/chinese-ai-writing-share/blob/main/Inference_T5_Finetune_Chinese_Couplet_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference for models trained from [T5 chinese couplet colab](https://github.com/hululuzhu/chinese-ai-writing-share/blob/main/Mengzi_T5_Finetune_Chinese_Couplet_V1.ipynb) and [T5 chinese Poem colab](https://github.com/hululuzhu/chinese-ai-writing-share/blob/main/WIP_Mengzi_T5_Finetune_Chinese_Poem_Writing_V1.ipynb)
- Download my saved models at [couplet model link](https://drive.google.com/drive/folders/1bQb_nrHHLkDYj09P2rrX7PSvHD8h3cTx?usp=sharing) and [poem model link](https://drive.google.com/drive/folders/1ZymaSbOcwlslD5tuUIk_9__C2dUJK_UY?usp=sharing)

## Load package and previously trained models

In [1]:
# Quite install simple T5 package
!pip install -q simplet5 &> /dev/null

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir -p my_t5/couplet
!mkdir -p my_t5/poem
!cp /content/drive/MyDrive/ML/Models/t5-couplet/simplet5-epoch-2-train-loss-3.126/* my_t5/couplet
!cp /content/drive/MyDrive/ML/Models/t5-poem/simplet5-epoch-2-train-loss-3.8297/* my_t5/poem

In [4]:
import torch
from simplet5 import SimpleT5
from transformers import T5Tokenizer, T5ForConditionalGeneration

class MengziSimpleT5(SimpleT5):
  def __init__(self) -> None:
    super().__init__()
    self.device = torch.device("cuda")

  def load_my_model(self, local_path, use_gpu: bool = True):
    self.tokenizer = T5Tokenizer.from_pretrained("Langboat/mengzi-t5-base")
    self.model = T5ForConditionalGeneration.from_pretrained(local_path)

Global seed set to 42


In [23]:
couplet_model = MengziSimpleT5()
couplet_model.load_my_model(local_path='my_t5/couplet')
couplet_model.model = couplet_model.model.to('cuda')

COUPLET_PROMPOT = '对联：'
MAX_SEQ_LEN = 32
MAX_OUT_TOKENS = MAX_SEQ_LEN

def couplet(in_str, model=couplet_model):
  model.model = model.model.to('cuda')
  in_request = f"{COUPLET_PROMPOT}{in_str[:MAX_SEQ_LEN]}"
  print(f"上： {in_str}\n下：", model.predict(
      in_request,
      max_length=MAX_OUT_TOKENS)[0])

In [16]:
AUTHOR_PROMPT = "模仿："
TITLE_PROMPT = "作诗："
EOS_TOKEN = '</s>'

poem_model = MengziSimpleT5()
poem_model.load_my_model(local_path='my_t5/poem')
poem_model.model = poem_model.model.to('cuda')
MAX_AUTHOR_CHAR = 4
MAX_TITLE_CHAR = 12
MIN_CONTENT_CHAR = 10
MAX_CONTENT_CHAR = 64

def poem(title_str, opt_author=None, model=poem_model):
  model.model = model.model.to('cuda')
  if opt_author:
    in_request = TITLE_PROMPT + title_str[:MAX_TITLE_CHAR] + EOS_TOKEN + AUTHOR_PROMPT + opt_author[:MAX_AUTHOR_CHAR]
  else:
    in_request = TITLE_PROMPT + title_str[:MAX_TITLE_CHAR]
  print(f"标题： {in_request}\n诗歌：", model.predict(
      in_request,
      max_length=MAX_CONTENT_CHAR)[0])

## Inference now
- Note we turned off sampling to see determistic results for comparison

In [25]:
# epoch 3 after 6 hours, looks good enough
for pre in ['欢天喜地度佳节', '不待鸣钟已汗颜，重来试手竟何艰',
            '当年欲跃龙门去，今日真披马革还', '载歌在谷',
            '北国风光，千里冰封，万里雪飘',
            '独立寒秋，湘江北去，橘子洲头']:
  couplet(pre)

上： 欢天喜地度佳节
下： 笑语欢歌迎新春
上： 不待鸣钟已汗颜，重来试手竟何艰
下： 何堪击鼓频催泪?一别伤心更枉然
上： 当年欲跃龙门去，今日真披马革还
下： 此日当登虎榜来,他年又见龙图新
上： 载歌在谷
下： 对酒当歌
上： 北国风光，千里冰封，万里雪飘
下： 南疆气象,五湖浪涌,三江潮来
上： 独立寒秋，湘江北去，橘子洲头
下： 遥思故里,雁字南飞,橘子洲头


In [26]:
for title in ['秋思', '百度', "百花", "谷歌", "中秋"]:
  for author in ['', "杜甫", "李白", "李清照", "苏东坡"]:
    poem(title, author)
  print()

标题： 作诗：秋思
诗歌： 秋色满西风,萧萧客思中。雁声连远水,砧韵入疏桐。病骨愁边减,衰顔镜裏空。故园归未得,何日赋归鸿。
标题： 作诗：秋思</s>模仿：杜甫
诗歌： 西风动高树,秋色满江城。客思惊残夢,乡心寄远情。雁声随暮角,砧韵入寒更。欲问南飞翼,云山隔此生。
标题： 作诗：秋思</s>模仿：李白
诗歌： 秋色满西楼,凉风动客愁。雁声连海远,人影入江流。月落千山静,天高万籁秋。何当一樽酒,相与醉高楼。
标题： 作诗：秋思</s>模仿：李清照
诗歌： 秋色满庭树,萧萧风露寒。客心随雁远,归夢绕江干。月落千山静,天高万里宽。故园何日到,一叶下云端。
标题： 作诗：秋思</s>模仿：苏东坡
诗歌： 秋色已萧索,客愁还郁陶。雨声来枕上,月色到樽中。老去心犹壮,闲来眼更空。故园归未得,何日是西风。

标题： 作诗：百度
诗歌： 百尺危楼倚碧空,一川秋色晴空。不知身在青天外,何似云间见日中。
标题： 作诗：百度</s>模仿：杜甫
诗歌： 蜀道多险阻,秦川多崄。云深迷鸟道,雨急失龙湫。地僻人稀到,天高日易秋。何当乘兴去,一叶泛沧洲。
标题： 作诗：百度</s>模仿：李白
诗歌： 蜀道如天险,秦川似掌平。三山连海尽,一水到江清。
标题： 作诗：百度</s>模仿：李清照
诗歌： 百尺危楼倚碧空,千寻古木锁晴空。行人莫道无仙骨,只合乘云到阆风。
标题： 作诗：百度</s>模仿：苏东坡
诗歌： 百尺危楼倚碧空,千寻古木锁晴空。不知何处飞来雨,但见江声到枕中。

标题： 作诗：百花
诗歌： 百花开尽绿阴成,春色何曾减旧情。自是东风多意思,不教风雨损花英。
标题： 作诗：百花</s>模仿：杜甫
诗歌： 百花春色好,花发满江城。日暖游蜂闹,风和戏蝶轻。乱飞红踯躅,争舞翠旌。莫怪频回首,芳菲已暗生。
标题： 作诗：百花</s>模仿：李白
诗歌： 百花如锦水,春色满芳洲。日暖游鱼乐,风和戏蝶愁。
标题： 作诗：百花</s>模仿：李清照
诗歌： 百花开尽绿阴成,春色何曾减旧情。莫道东风无意思,只应留得两三声。
标题： 作诗：百花</s>模仿：苏东坡
诗歌： 百花开尽绿阴成,春色都归醉眼明。莫道东风无意思,只应留得两三声。

标题： 作诗：谷歌
诗歌： 西山有奇石,上有千丈松。下有百尺松,上无千岁松。
标题： 作诗：谷歌</s>模仿：杜甫
诗歌： 巴西有奇石,磊落不可攀。上有百尺松,下有千岁山